In [ ]:
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import ChatPromptTemplate
from langchain_openai import ChatOpenAI
from langchain_core.tools import tool

import os
os.environ["OPEN_API_KEY"] = "your key"


@tool
def build(commit_id: int):
    """build artifact with commit_id ."""
    print(f"Building artifact with commit_id {commit_id}")
    return 

@tool
def deploy(build_id: int):
    """deploy artifact with build_id. """
    print(f"Deploying artifact with build_id {build_id}")
    return 

@tool
def test(deploy_id: int):
    "test artifact with deploy_id."
    print(f"Testing artifact with deploy_id {deploy_id}")
    return 

#build.invoke({"commit_id": 1})

from operator import itemgetter
from typing import Union

from langchain.output_parsers import JsonOutputToolsParser
from langchain_core.runnables import (
    Runnable,
    RunnableLambda,
    RunnableMap,
    RunnablePassthrough,
)

model = ChatOpenAI(openai_api_key=os.environ["OPEN_API_KEY"], model_name="gpt-4-0125-preview")

tools = [build, deploy, test]



In [ ]:
from langchain_core.prompts.chat import (
    ChatPromptTemplate,
    HumanMessagePromptTemplate,
    MessagesPlaceholder,
)
# Get the prompt to use 
# https://github.com/hwchase17/langchain-hub?tab=readme-ov-file
#prompt = load_prompt('lc://prompts/conversation/prompt.json')

prompt = ChatPromptTemplate.from_messages(
    [
        ("system", "You are a helpful assistant."),
        MessagesPlaceholder(
            variable_name="chat_history",optional=True
        ),
        ("human", "{input}"),
        MessagesPlaceholder(
            variable_name="agent_scratchpad",
        ),
    ]
)
prompt.messages
#prompt.pretty_print()

In [ ]:
from langchain.agents import AgentExecutor, create_openai_tools_agent

# Construct the OpenAI Tools agent
agent = create_openai_tools_agent(model, tools, prompt)



In [ ]:
agent_executor = AgentExecutor(agent=agent, tools=tools, verbose=True)

agent_executor.invoke(
    {
        "input": "ninety divided by three, multiply by the sum of twelve and three, then square the whole result"
    }
)